In [1]:
%load_ext autoreload
%autoreload 2

# data

In [2]:
with open("./tokens.txt") as fin:
    tokens = filter(len,fin.read().split('\n'))
    
tokens.append("_BOS_") #beginning of sentence. Omitted in danet
tokens.append("_PAD_") #padding. Omitted in danet

UNK_ix,BOS_ix,EOS_ix,PAD_ix = map(tokens.index,["_UNK_","_BOS_","_EOS_","_PAD_"])
n_tokens = len(tokens)

from collections import defaultdict
token_to_ix = defaultdict(lambda:UNK_ix,{t:i for i,t in enumerate(tokens)})

In [3]:
with open("./sp.speakers.c5.txt.val.predictions") as fin:
    probas = map(lambda line:map(float,line.split()),fin)
    
with open("./sp.speakers.c5.txt.val") as fin:
    phrases,answers = zip(*map(lambda line: line.split('\t'),fin))
    

In [4]:
!head sp.speakers.c5.txt.val

oh my god i can ' t believe it ! ! ! _EOS_ hello hello , are you still there ? _EOS_ they want to sign me to the _UNK_ _UNK_ ! _EOS_ you want to sign ms . lopez ? _EOS_ oh my god , it ' s happening for me ! _EOS_ _Cartman_	yes ! yes , we can record an album next week ! sure we can write ten songs ! we ' ll start tonight ! ! _EOS_
they want to sign me to the _UNK_ _UNK_ ! _EOS_ you want to sign ms . lopez ? _EOS_ oh my god , it ' s happening for me ! _EOS_ yes ! yes , we can record an album next week ! sure we can write ten songs ! we ' ll start tonight ! ! _EOS_ we ' re gonna be _UNK_ ! _EOS_ _Cartman_	come on , we gotta get to work on some songs ! _EOS_
_EOS_ _Cartman_	mr . broflovski ! mr . broflovski , there a smug storm ! we have to go ! _EOS_
_EOS_ mr . broflovski ! mr . broflovski , there a smug storm ! we have to go ! _EOS_ being smug is a good thing . _EOS_ _Cartman_	oh my christ ! _EOS_
mr . broflovski ! mr . broflovski , there a smug storm ! we have to go ! _EOS_ being smug i

In [5]:
import numpy as np
assert np.allclose(
    map(len,probas),
    map(len,map(str.split,answers))
    )

In [6]:

def preprocess(line):
    return map(token_to_ix.__getitem__,line.split())

def phrase_to_matrix(phrases,max_len = None,PAD_ix=PAD_ix):
    
    phrases_ix = map(preprocess,phrases)
    max_len = max_len or max(map(len,phrases_ix))
    
    matrix = np.zeros((len(phrases),max_len),dtype='int32') + PAD_ix
    
    for i,phrase_ix in enumerate(phrases_ix):
        matrix[i,:min(len(phrase_ix),max_len)] = phrase_ix[:max_len]
        
    return matrix



# model

In [7]:
model_path = "weights/"
grad_clipping=5

In [8]:
import numpy as np
import os
def read_wmtx(fname,dtype='float32',keepdims=False):
    "reads matrix/vector from text file"
    with open(fname) as fin:
        rows = fin.read().split('\n')[1:-1]        
        rows = map(lambda row: map(float,filter(len,row.split(' '))),rows)
        if not keepdims:
            while len(rows)==1:
                rows=rows[0]
        return np.array(rows,dtype='float32')


In [9]:
#read shared embedding matrix

danet_emb_W = read_wmtx(os.path.join(model_path,"layer_context_embedding_0.wmtx"))
danet_emb_b = read_wmtx(os.path.join(model_path,"layer_context_embedding_biases.wmtx"))

#ACTUAL embedding W - add b and append b for BOS
emb_W = np.concatenate((danet_emb_W+danet_emb_b[None,:],
                        danet_emb_b[None,:],#BOS
                        danet_emb_b[None,:],#PAD
                       ),axis=0)

#assert np.all(emb_W[BOS_ix]== danet_emb_b)

In [10]:
import glob

def load_LSTMLayer_from_danet(lstm,path=model_path,prefix="layer_context_lstm",verbose=False):
    
    #count lstm inputs, exclude recurrent weight and b
    n_inputs = len(glob.glob(os.path.join(path,prefix+'_pre_cell_value_*')))-2 
    inputs = range(n_inputs) #[matrices corresponding to non-recurrent links]
    hid = n_inputs #id of hidden->hidden connections
    
    conversion_dict= {

        lstm.W_in_to_cell:[prefix+'_pre_cell_value_%i.wmtx'%i for i in inputs],
        lstm.W_hid_to_cell:[prefix+'_pre_cell_value_%i.wmtx'%hid],
        lstm.b_cell:[prefix+'_pre_cell_value_biases.wmtx'],

        lstm.W_in_to_forgetgate:[prefix+'_pre_forget_gate_%i.wmtx'%i for i in inputs],
        lstm.W_hid_to_forgetgate:[prefix+'_pre_forget_gate_%i.wmtx'%hid],
        lstm.b_forgetgate: [prefix+'_pre_forget_gate_biases.wmtx'],

        lstm.W_in_to_ingate:[prefix+'_pre_input_gate_%i.wmtx'%i for i in inputs],
        lstm.W_hid_to_ingate:[prefix+'_pre_input_gate_%i.wmtx'%hid],
        lstm.b_ingate:[prefix+'_pre_input_gate_biases.wmtx'],

        lstm.W_in_to_outgate:[prefix+'_pre_output_gate_%i.wmtx'%i for i in inputs],
        lstm.W_hid_to_outgate:[prefix+'_pre_output_gate_%i.wmtx'%hid],
        lstm.b_outgate:[prefix+'_pre_output_gate_biases.wmtx'],
    }
    for param,fnames in conversion_dict.items():
        matrices = []
        for fname in fnames:
            fpath = os.path.join(path,fname)
            wmtx = read_wmtx(fpath)
            matrices.append(wmtx)
        full_wmtx = np.concatenate(matrices,0)
        if verbose:
            print "loading",','.join(fnames),'shape=',full_wmtx.shape,
            print "as",param.name,param.get_value().shape
        assert full_wmtx.shape == param.get_value().shape
        param.set_value(full_wmtx)
    return lstm

In [11]:
from warnings import warn

import theano
import theano.tensor as T
from wrong_lstm_layer import WrongLSTMLayer
from lasagne.layers import *

class encoder:
        
    input_phrase  = T.imatrix("encoder phrase tokens")
    
    #bos_column = T.zeros((input_phrase.shape[0],1),'int32')+BOS_ix
    #input_phrase_bos = T.concatenate((bos_column,input_phrase),axis=1)  #prepend BOS
    warn("i did not prepend BOS in the encoder. Am i right?")
    
    l_in = InputLayer((None,None),input_phrase,name='context input')
    l_mask = InputLayer((None,None),T.neq(input_phrase,PAD_ix),'context mask')
    
    l_emb = EmbeddingLayer(l_in,n_tokens,emb_W.shape[1], 
                             W=emb_W,name="context embedding")
    
    
        
    ####LSTMLayer with incorrect outputgate####
        
    lstm_units = read_wmtx(os.path.join(model_path,"layer_context_lstm_pre_forget_gate_biases.wmtx")).shape[0]

    
    l_lstm = WrongLSTMLayer(
                        l_emb,
                        lstm_units,
                        name='encoder_lstm',
                        grad_clipping=grad_clipping,
                        mask_input = l_mask,
                        only_return_final=True,
                        peepholes=False)
    
    l_lstm = load_LSTMLayer_from_danet(
                l_lstm,
                model_path,
                'layer_context_lstm')
    
    
    output = l_lstm
        


        
        


/home/jheuristic/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:14: UserWarning: i did not prepend BOS in the encoder. Am i right?


In [12]:
from agentnet.memory import LSTMCell

In [15]:
def load_LSTMdecoder_from_danet(params,lstm_name,path=model_path,prefix="layer_reply_lstm",verbose=False):
    
    emb,encoder,out = range(3)
    
    conversion_dict = {
         "%s.b_to_ingate"%(lstm_name):prefix+'_pre_input_gate_biases.wmtx',
         "%s.W_out_to_ingate"%(lstm_name):prefix+'_pre_input_gate_%i.wmtx'%out,
         "%s.W_emb_to_ingate"%(lstm_name):prefix+'_pre_input_gate_%i.wmtx'%emb,
         "%s.W_encoder_to_ingate"%(lstm_name):prefix+'_pre_input_gate_%i.wmtx'%encoder,
         "%s.b_to_forgetgate"%(lstm_name):prefix+'_pre_forget_gate_biases.wmtx',
         "%s.W_out_to_forgetgate"%(lstm_name):prefix+'_pre_forget_gate_%i.wmtx'%out,
         "%s.W_emb_to_forgetgate"%(lstm_name):prefix+'_pre_forget_gate_%i.wmtx'%emb,
         "%s.W_encoder_to_forgetgate"%(lstm_name):prefix+'_pre_forget_gate_%i.wmtx'%encoder,
         "%s.b_to_cell"%(lstm_name):prefix+'_pre_cell_value_biases.wmtx',
         "%s.W_out_to_cell"%(lstm_name):prefix+'_pre_cell_value_%i.wmtx'%out,
         "%s.W_emb_to_cell"%(lstm_name):prefix+'_pre_cell_value_%i.wmtx'%emb,
         "%s.W_encoder_to_cell"%(lstm_name):prefix+'_pre_cell_value_%i.wmtx'%encoder,
         "%s.b_to_outgate"%(lstm_name):prefix+'_pre_output_gate_biases.wmtx',
         "%s.W_out_to_outgate"%(lstm_name):prefix+'_pre_output_gate_%i.wmtx'%out,
         "%s.W_emb_to_outgate"%(lstm_name):prefix+'_pre_output_gate_%i.wmtx'%emb,
         "%s.W_encoder_to_outgate"%(lstm_name):prefix+'_pre_output_gate_%i.wmtx'%encoder,
    }
    
    param_dict = {param.name:param for param in params}
    assert len(param_dict) == len(params)
    
    
    for param_name in conversion_dict:
        
        
        fpath = os.path.join(path,conversion_dict[param_name])
        wmtx = read_wmtx(fpath)
            
        if verbose:
            print "loading",conversion_dict[param_name],'shape=',wmtx.shape,
            print "as",param_name,param_dict[param_name].get_value().shape
        assert wmtx.shape == param_dict[param_name].get_value().shape

        param_dict[param_name].set_value(wmtx)



In [78]:
from wrong_lstm_cell import WrongLSTMCell
from agentnet import Recurrence
from agentnet.resolver import  ProbabilisticResolver
class decoder:
    lstm_units = encoder.output.output_shape[1]
    
    prev_cell = InputLayer((None,lstm_units),name='cell')
    prev_out = InputLayer((None,lstm_units),name='out')
    
    #input
    inp_word = InputLayer((None,))
    word_embedding = EmbeddingLayer(inp_word,n_tokens,emb_W.shape[1],
                                         W=encoder.l_emb.W,name='emb')
    encoder_lstm = InputLayer((None,lstm_units),name='encoder')
    
    #recurrent units
    new_cell,new_out = WrongLSTMCell(prev_cell,prev_out,
                                     input_or_inputs=[word_embedding,encoder_lstm],
                                     name='decoder_lstm',peepholes=False
                                    )
    
    load_LSTMdecoder_from_danet(get_all_params(new_out,trainable=True),'decoder_lstm',verbose=True)

    #predicting output
    W_bottleneck = read_wmtx(os.path.join(model_path,"layer_fc_intermediate_decoder_0.wmtx"))
    b_bottleneck = read_wmtx(os.path.join(model_path,"layer_fc_intermediate_decoder_biases.wmtx"))        
    bottleneck_units = W_bottleneck.shape[-1]

    bottleneck = DenseLayer(new_out,bottleneck_units,
                              nonlinearity=T.tanh,
                              W = W_bottleneck,
                              b = b_bottleneck,
                              name='decoder intermediate')

    W_output = read_wmtx(os.path.join(model_path,"layer_fc_decoder_0.wmtx"))
    b_output = read_wmtx(os.path.join(model_path,"layer_fc_decoder_biases.wmtx"))        
    #add two remaining tokens: BOS and PAD with large negative b to minimize influence
    W_output = np.concatenate([W_output,np.zeros_like(W_output[:,:2])],-1)
    b_output = np.concatenate([b_output,np.zeros_like(b_output[:2])-100],-1)
    
    
    next_word_probs = DenseLayer(bottleneck,n_tokens,
                                   nonlinearity = T.nnet.softmax,
                                   W = W_output,
                                   b = b_output,
                                 name='decoder next word probas')

    next_words = ProbabilisticResolver(next_word_probs,assume_normalized=True)

loading layer_reply_lstm_pre_forget_gate_biases.wmtx shape= (1024,) as decoder_lstm.b_to_forgetgate (1024,)
loading layer_reply_lstm_pre_cell_value_biases.wmtx shape= (1024,) as decoder_lstm.b_to_cell (1024,)
loading layer_reply_lstm_pre_forget_gate_2.wmtx shape= (1024, 1024) as decoder_lstm.W_out_to_forgetgate (1024, 1024)
loading layer_reply_lstm_pre_input_gate_1.wmtx shape= (1024, 1024) as decoder_lstm.W_encoder_to_ingate (1024, 1024)
loading layer_reply_lstm_pre_input_gate_biases.wmtx shape= (1024,) as decoder_lstm.b_to_ingate (1024,)
loading layer_reply_lstm_pre_input_gate_2.wmtx shape= (1024, 1024) as decoder_lstm.W_out_to_ingate (1024, 1024)
loading layer_reply_lstm_pre_forget_gate_1.wmtx shape= (1024, 1024) as decoder_lstm.W_encoder_to_forgetgate (1024, 1024)
loading layer_reply_lstm_pre_output_gate_1.wmtx shape= (1024, 1024) as decoder_lstm.W_encoder_to_outgate (1024, 1024)
loading layer_reply_lstm_pre_forget_gate_0.wmtx shape= (512, 1024) as decoder_lstm.W_emb_to_forgetgate (

In [79]:
class trainer:
    #training recurrence
    reference_answers = T.imatrix("decoder reference answers")
    bos_column = T.zeros((reference_answers.shape[0],1),'int32')+BOS_ix
    reference_answers_bos = T.concatenate((bos_column,reference_answers ),axis=1)  #prepend BOS
        
    l_ref = InputLayer((None,None),reference_answers_bos,name='context input')
    l_ref_mask = InputLayer((None,None),T.neq(reference_answers_bos,PAD_ix),'context mask')

    recurrence = Recurrence(input_sequences={decoder.inp_word:l_ref},
                           input_nonsequences={decoder.encoder_lstm:encoder.output},
                           state_variables={decoder.new_cell:decoder.prev_cell,
                                            decoder.new_out:decoder.prev_out},
                           tracked_outputs=[decoder.next_word_probs],
                           mask_input=l_ref_mask,
                           unroll_scan=False,)
    
    
    
    out = get_output(recurrence[decoder.next_word_probs])
    
    likelihood = out[T.arange(out.shape[0])[:,None],T.arange(out.shape[1]-1)[None,:], reference_answers]
    
    predict_proba = theano.function([encoder.input_phrase,reference_answers],likelihood)
    

/home/jheuristic/Downloads/AgentNet/agentnet/agent/recurrence.py:188: UserWarning: State_variables recommended type is OrderedDict.
                Otherwise, order of agent state outputs from get_sessions and get_agent_reaction methods
                may depend on python configuration.

                Current order is: [<lasagne.layers.merge.ElemwiseMergeLayer object at 0x7f37ace39810>, <lasagne.layers.special.NonlinearityLayer object at 0x7f37ace395d0>]
                You may find OrderedDict in standard collections module: from collections import OrderedDict
                
  """.format(state_variables=list(self.state_variables.keys())))


In [80]:
class generator:
    
    n_steps = T.iscalar()
    bos_input_layer = InputLayer((None,),T.zeros((encoder.input_phrase.shape[0],),'int32')+BOS_ix, name="first input")

    recurrence = Recurrence(
                           input_nonsequences={decoder.encoder_lstm:encoder.output},
                           state_variables={decoder.new_cell:decoder.prev_cell,
                                            decoder.new_out:decoder.prev_out,
                                            decoder.next_words:decoder.inp_word},
                           tracked_outputs=[decoder.next_words],
                           state_init={decoder.next_words:bos_input_layer},
                           n_steps=n_steps,
                           unroll_scan=False,)
    
    
    
    out = get_output(recurrence[decoder.next_words])
    
    
    
    generate = theano.function([encoder.input_phrase,n_steps],out,
                              updates = recurrence.get_automatic_updates())
    @staticmethod
    def reply(phrase,max_len=100):
        phrase_ix = phrase_to_matrix([phrase])
        answer_ix = generator.generate(phrase_ix,max_len)[0]
        if EOS_ix in answer_ix:
            answer_ix = answer_ix[:list(answer_ix).index(EOS_ix)]
        return ' '.join(map(tokens.__getitem__, answer_ix))
        

/home/jheuristic/Downloads/AgentNet/agentnet/agent/recurrence.py:188: UserWarning: State_variables recommended type is OrderedDict.
                Otherwise, order of agent state outputs from get_sessions and get_agent_reaction methods
                may depend on python configuration.

                Current order is: [<lasagne.layers.merge.ElemwiseMergeLayer object at 0x7f37ace39810>, <lasagne.layers.special.NonlinearityLayer object at 0x7f37ace395d0>, <agentnet.resolver.probabilistic.ProbabilisticResolver object at 0x7f37aeabf450>]
                You may find OrderedDict in standard collections module: from collections import OrderedDict
                
  """.format(state_variables=list(self.state_variables.keys())))


In [81]:
from agentnet.utils.persistence import save,load
save(generator.recurrence,"cartmanez_from_danet.pcl")

In [72]:
generator.reply(" how are you doing ? ",10)

"i ' m all right ."

In [41]:
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline 
probs_predicted = []
for i in tqdm(range(len(answers))):
    print 'A:', phrases[i]
    print 'B:', answers[i]
    probs_predicted.append(predict_proba(phrase_to_matrix(phrases[i:i+1]),phrase_to_matrix(answers[i:i+1]))[0])
    plt.plot(probs_predicted[i],label='pred',marker='+')
    plt.plot(probas[i],label='ref',marker='+')
    plt.yscale('log')
    plt.legend()
    plt.grid()
    plt.show()
    
from scipy.stats import spearmanr,pearsonr
print spearmanr(probs_predicted[0],probas[0])
print pearsonr(probs_predicted[0],probas[0])

  0%|          | 0/432 [00:00<?, ?it/s]

A: oh my god i can ' t believe it ! ! ! _EOS_ hello hello , are you still there ? _EOS_ they want to sign me to the _UNK_ _UNK_ ! _EOS_ you want to sign ms . lopez ? _EOS_ oh my god , it ' s happening for me ! _EOS_ _Cartman_
B: yes ! yes , we can record an album next week ! sure we can write ten songs ! we ' ll start tonight ! ! _EOS_



KeyboardInterrupt: 

# trash line

In [ ]:
from theano import tensor as T
from lasagne.layers import *
import os


#embedding

emb_B = read_wmtx(os.path.join(enc_path,"layer_context_embedding_biases.wmtx"))#!unused
emb_W = read_wmtx(os.path.join(enc_path,"layer_context_embedding_0.wmtx"))
n_tokens,word_embedding_size = emb_W.shape

prev_phrase = T.imatrix('input tokens')
#mask-out the PAD tokens for short phrases
prev_phrase_mask = T.neq(prev_phrase,1)#PAD token,)


In [ ]:
read_wmtx(os.path.join(enc_path, 'layer_reply_lstm_pre_cell_value_0.wmtx',)).shape


In [ ]:
gru0_units=1024
grad_clip=5
decoder_bottleneck_size = 256

In [ ]:

import lasagne
from wrong_lstm_cell import WrongLSTMCell
from agentnet.resolver import ProbabilisticResolver
###one-step update of decoder recurrence. Used for both training and generation.
class decoder_step:
    

    prev_word = InputLayer((None,),name='decoder prev output inp')

    prev_output_emb = EmbeddingLayer(prev_word,
                                     n_tokens,
                                     word_embedding_size,
                                     W= encoder.l_emb.W,
                                     name='dec input 0')
    
    encoder_last_state_inp = InputLayer((None,gru0_units),name='dec input 1')

    #previous GRU state goes here
    prev_lstm_cell = InputLayer([None,gru0_units],name='decoder prev lstm cell')
    prev_lstm_hid = InputLayer([None,gru0_units],name='decoder prev lstm hid')
    

    #gru update
    lstm_cell,lstm_hid = WrongLSTMCell(prev_lstm_cell,prev_lstm_hid,[prev_output_emb,encoder_last_state_inp],name='dec lstm',peepholes=False)

    
    l_intermediate = DenseLayer(lstm_hid,256,nonlinearity=lasagne.nonlinearities.tanh,name='decoder intermediate')
    
    #predicted probabilities of next word -- with temperature applied
    next_word_probas = DenseLayer(l_intermediate,n_tokens,
                                   nonlinearity=lambda x: lasagne.nonlinearities.softmax(x*greed),
                                   #softmax with temperature
                                   name='decoder next letter probas')
    

    #actual next word picked with output probabilities
    next_word = ProbabilisticResolver(next_word_probas,
                                       assume_normalized=True,
                                       name='decoder next letter picker')
    



In [ ]:
sorted({par.name:par for par in encoder.l_lstm.get_params()})

In [ ]:
filter(lambda n: n not in conversion_dict.values(), matrix_names)[:10]

In [ ]:
# 0        1          2          3
#ingate,forgetgate,cell input, outgate




In [ ]:
import numpy as np
def read_wmtx(fname,dtype='float32',keepdims=False):
    with open(fname) as fin:
        
        rows = fin.read().split('\n')[1:-1]        
        
        rows = map(lambda row: map(float,filter(len,row.split(' '))),rows)
        
        if not keepdims:
            while len(rows)==1:
                rows=rows[0]
        
        return np.array(rows,dtype='float32')


channels={'chl0':'ingate',
   'chl1':'forgetgate',
   'chl2':'cell',
   'chl3':'outgate'
   }
for par in get_all_params(decoder_step.next_word):
    if any(par.name.endswith(sfx) for sfx in channels):
        par.name = par.name[:-4]+channels[par.name[-4:]]

dec_params = {par.name:par for par in get_all_params(decoder_step.next_word)}




    
conversion_dict = {encoder.l_emb.W:"layer_context_embedding_0.wmtx",}

conversion_dict.update(convert_lstm(encoder.l_lstm,"layer_context_lstm"))


conversion_dict.update({
    dec_params[k]:v for k,v in {
            'decoder intermediate.W':'layer_fc_intermediate_decoder_0.wmtx',
            'decoder intermediate.b':'layer_fc_intermediate_decoder_biases.wmtx',
            'decoder next letter probas.W':'layer_fc_decoder_0.wmtx',
            'decoder next letter probas.b':'layer_fc_decoder_biases.wmtx',
            
            'dec lstm.gates.W_dec input 0_cell':'layer_reply_lstm_pre_cell_value_0.wmtx',
            'dec lstm.gates.W_dec input 1_cell':'layer_reply_lstm_pre_cell_value_1.wmtx',
            'dec lstm.gates.W_decoder prev lstm hid_cell':'layer_reply_lstm_pre_cell_value_2.wmtx',
            'dec lstm.gates.b_cell':'layer_reply_lstm_pre_cell_value_biases.wmtx',
            
            'dec lstm.gates.W_dec input 0_forgetgate':'layer_reply_lstm_pre_forget_gate_0.wmtx',
            'dec lstm.gates.W_dec input 1_forgetgate':'layer_reply_lstm_pre_forget_gate_1.wmtx',
            'dec lstm.gates.W_decoder prev lstm hid_forgetgate':'layer_reply_lstm_pre_forget_gate_2.wmtx',
            'dec lstm.gates.b_forgetgate':'layer_reply_lstm_pre_forget_gate_biases.wmtx',
            
            'dec lstm.gates.W_dec input 0_ingate':'layer_reply_lstm_pre_input_gate_0.wmtx',
            'dec lstm.gates.W_dec input 1_ingate':'layer_reply_lstm_pre_input_gate_1.wmtx',
            'dec lstm.gates.W_decoder prev lstm hid_ingate':'layer_reply_lstm_pre_input_gate_2.wmtx',
            'dec lstm.gates.b_ingate':'layer_reply_lstm_pre_input_gate_biases.wmtx',
            
            
            'dec lstm.gates.W_dec input 0_outgate':'layer_reply_lstm_pre_output_gate_0.wmtx',
            'dec lstm.gates.W_dec input 1_outgate':'layer_reply_lstm_pre_output_gate_1.wmtx',
            'dec lstm.gates.W_decoder prev lstm hid_outgate':'layer_reply_lstm_pre_output_gate_2.wmtx',
            'dec lstm.gates.b_outgate':'layer_reply_lstm_pre_output_gate_biases.wmtx'

        }.items()
    })


for k,v in conversion_dict.items():
    assert tuple(k.shape.eval()) == read_wmtx(os.path.join(enc_path,v)).shape

In [ ]:
### Use the same decoder to generate tokens.
# Do this by feeding it's output back as it's input
from collections import OrderedDict

class generative:


    recurrent_states = OrderedDict({
            decoder_step.l_gru0:decoder_step.l_prev_gru0,
            decoder_step.next_word:decoder_step.prev_word
        })
    


    recurrence = Recurrence(state_variables=recurrent_states,
                            state_init = {decoder_step.l_gru0 : encoder.output},
                            tracked_outputs=(decoder_step.next_word_probas,decoder_step.next_word),
                            batch_size=batch_size,
                            n_steps = max_len,
                            unroll_scan=unroll_scan,
                            )



    state_seqs, (probas_seq,output_tokens_seq) = recurrence.get_sequence_layers()

    reply_tokens = lasagne.layers.get_output(output_tokens_seq)
    
    apply_fun = theano.function([prev_phrase],reply_tokens,
                                 updates=recurrence.get_automatic_updates())

In [ ]:
from agentnet.memory import GRUCell
from agentnet.resolver import ProbabilisticResolver
from agentnet.agent import Recurrence




In [ ]:
rows[0]

In [ ]:
lines[-1]